In [6]:
#####split data#####
#install.packages("devtools")
#devtools::install_github("stephenturner/annotables")
library(annotables)
library(plyr)

In [7]:
adjustdir = function(data){
  levels(data[,1]) = union(levels(data[,1]),levels(data[,2]))
  levels(data[,2]) = levels(data[,1])
  for (i in (1:dim(data)[1])){
    if (data[i,4] == '<-'){
      a = data[i,1]
      data[i,1] = data[i,2]
      data[i,2] = a
      data[i,4] = '->'
    }
    if (data[i,4] == '|-'){
      b = data[i,1]
      data[i,1] = data[i,2]
      data[i,2] = b
      data[i,4] = '-|'
      
    }
  }
  datag = data[,1:2]
  colnames(datag) = c('gene0','gene1')
  return(datag)
}

addid = function(data){
  ID = grch38
  data = data[,1:2]
  colnames(data) = c('symbol','gene1')
  dataID1 = merge(ID[,2:3], data)
  colnames(dataID1) = c('gene0','ID0', 'symbol')
  dataID2 = merge(ID[,2:3],dataID1)
  dataID2 = dataID2[,c(2,4)]
  colnames(dataID2) = c('ID0','ID1')
  return(dataID2)
}

splitnormdata = function(data){
  gene0 = as.character(data$name)
  gene1 = as.character(data$name)
  for (i in (1:dim(data)[1])){
    gene0[i] = substring(gene0[i], 1, regexpr('(pp)', gene0[i])-3)
    gene1[i] = substring(gene1[i], regexpr('(pp)', gene1[i])+4,nchar(gene1[i]))
  }
  
  gene0 = data.frame(gene0)
  gene1 = data.frame(gene1)
  Et = cbind(gene0, gene1)
  data = Et
  return(data)
}


sta = function(datag, normedgeg){
  dataco = datag[datag[,1]%in%intersect(normedgeg[,1],datag[,1]),]
  datacoo = dataco[dataco[,2]%in%intersect(normedgeg[,2],datag[,2]),]
  normedgegco = normedgeg[normedgeg[,1]%in%intersect(normedgeg[,1],union(datag[,1], datag[,2])),]
  normedgegcoo = normedgegco[normedgegco[,2]%in%intersect(normedgeg[,2],union(datag[,1],datag[,2])),]
  edges = dim(datacoo)[1]
  rightedges = dim(normedgegcoo)[1]
  node = max(length(unique(datacoo[,1])),length(unique(datacoo[,2])))
  mean = mean(count(datacoo[,2])[,2])
  max = max(count(datacoo[,2])[,2])
  varify = merge(datacoo[,1:2], normedgegcoo)
  right = dim(varify)[1]
  precision = right/edges
  recall = right/rightedges
  result = list(node,edges,rightedges , mean, max, right, precision, recall, varify, normedgegcoo)
  return(result)
}

In [9]:
#####pp interaction#####
normedge = read.csv('Right/PP/String Network default edge.csv')
normedge = normedge[normedge$score>0.5,]
normedgeg = splitnormdata(normedge)
normedgen = addid(normedgeg)

In [2]:
data = read.table('RB1Edge1.txt',header=F, skip=1)
datag = data[,c(2,4)]
colnames(datag) = c('gene0','gene1')
#write.table(datag, 'S_AD.csv', sep=',',col.names=F, row.names=F)
datan = addid(datag)
varify = sta(datag, normedgeg)

ERROR: Error in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, : line 1 did not have 4 elements


In [16]:
data = read.table('S_AD1.txt',header=F, skip=4)
datag = data[,c(2,4)]
colnames(datag) = c('gene0','gene1')
datagn = datag[-grep('deg:',datag$gene1),]
for (i in 1:dim(datagn)[1]){
    a = as.character(datagn[i,1])
    b = substring(a, 5,nchar(a))
    levels(datagn[,1]) = c(levels(datagn[,1]),b)
    datagn[i,1] = as.factor(b)
    c = as.character(datagn[i,2])
    d = substring(c, 5,nchar(c))
    levels(datagn[,2]) = c(levels(datagn[,2]),d)
    datagn[i,2] = as.factor(d)
}
varify = sta(datagn, normedgeg)
#write.csv(datagn,'RB11AD.csv', quote=F, row.names=F)

In [11]:
write.csv(varify[9],'Subgene1v.csv',quote=F, row.names=F)

ERROR: Error in is.data.frame(x): object 'varify' not found


In [ ]:
##################################################################################################################

In [ ]:
#####function impact#####
normedge = read.table(file.choose(),sep = '\t', header = 1)
normedgeg = adjustdir(normedge)

In [ ]:
data = read.csv(file.choose())
datag = data[,c(2,4)]
colnames(datag) = c('gene0','gene1')
varify = sta(datag, normedgeg)
write.csv(varify,'varify.csv',quote=F, row.names = F)